# Tạo Output thô sơ cho báo cáo chứng chỉ quỹ

In [1]:
# %%capture

import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)

from import_default import *
from import_database import *
from import_other import *

In [51]:
ctv_df = pd.read_excel('../data/ccq/cts02_danh_sach_ctv_ccq.xlsx')
cn_df = pd.read_excel('../data/ccq/cts03_danh_sach_cn_ccq.xlsx')

input_df = pd.read_excel('../data/ccq/cts01_phi_hop_tac_ccq.xlsx')
input_df = input_df[['account_no','investor_name','dksh','fund','trading_date','order_code','ccq_unit','nav','gross_amount','net_amount','rm_name','ma_cn_qltk','rm_code_vncapital','branch_code_vncapital']]
input_df = input_df.rename(columns={'order_code': 'txn_code'})

#### Xử lý OP1

In [49]:
def ten_cn_quan_ly_rm(code):
    try:
        branch_int = int(str(code)[-3:])
        return cn_df[cn_df['ma_cn'] == branch_int]['ten_cn'].iloc[0] if len(cn_df[cn_df['ma_cn'] == branch_int]) > 0 else np.nan
    except (ValueError, IndexError):
        return np.nan
    
def ten_cn_quan_ly_tktt(code):
    try:
        branch_int = int(str(code)[:3])
        return cn_df[cn_df['ma_cn'] == branch_int]['ten_cn'].iloc[0] if len(cn_df[cn_df['ma_cn'] == branch_int]) > 0 else np.nan
    except (ValueError, IndexError):
        return np.nan

In [ ]:
op01_chi_tiet_thang = input_df.copy()
op01_chi_tiet_thang['rm_name'] = op01_chi_tiet_thang['rm_name'].fillna("Dương Thuỳ Linh")
op01_chi_tiet_thang['rm_code_cts'] = op01_chi_tiet_thang['rm_code_vncapital'].map(ctv_df[['ma_dinh_danh_vnc', 'ma_ctv_tai_cts']].set_index('ma_dinh_danh_vnc')['ma_ctv_tai_cts'].to_dict()).fillna("CTS")
op01_chi_tiet_thang['branch_code_vncapital'] = op01_chi_tiet_thang['branch_code_vncapital'].fillna("CTS")
op01_chi_tiet_thang['ten_cn_quan_ly_rm'] = op01_chi_tiet_thang['branch_code_vncapital'].apply(ten_cn_quan_ly_rm)
op01_chi_tiet_thang['ten_cn_quan_ly_tktt'] = op01_chi_tiet_thang['ma_cn_qltk'].apply(ten_cn_quan_ly_tktt)

#### Xử lý OP2

#### Xử lý OP3

In [98]:
op03a_ctv_detail = op01_chi_tiet_thang.copy()
op03a_ctv_detail['phan_khuc'] = 'KHCN'
op03a_ctv_detail['chi_nhanh_vietinbank'] = op03a_ctv_detail['ten_cn_quan_ly_rm'].fillna(op03a_ctv_detail['ten_cn_quan_ly_tktt'])
op03a_ctv_detail['ma_cn'] = op03a_ctv_detail['chi_nhanh_vietinbank'].map(cn_df[['ma_cn', 'ten_cn']].set_index('ten_cn')['ma_cn'].to_dict())
op03a_ctv_detail['phi_dv_doi_tac_tra_cts'] = op03a_ctv_detail['net_amount'].apply(lambda x: 0.0065*x if x == 'VFF' else 0.0115*x)
op03a_ctv_detail['doanh_thu_ghi_nhan'] = op03a_ctv_detail.apply(lambda x: x['phi_dv_doi_tac_tra_cts']*0.7 if x['rm_code_cts'] == 'CTS' else x['phi_dv_doi_tac_tra_cts']*0.3, axis=1)
op03a_ctv_detail['vung'] = op03a_ctv_detail['ma_cn'].map(cn_df.set_index('ma_cn')['vung'].to_dict())

op03a_ctv_detail = op03a_ctv_detail[['account_no', 'investor_name', 'dksh', 'phan_khuc', 'rm_code_cts', 'ma_cn', 'chi_nhanh_vietinbank', 'net_amount', 'phi_dv_doi_tac_tra_cts', 'doanh_thu_ghi_nhan', 'vung']]

#### Xuất dữ liệu

In [101]:
with pd.ExcelWriter('../output/ccq/cts-ccq-report.xlsx', engine='openpyxl') as writer:
    op01_chi_tiet_thang.to_excel(writer, sheet_name='op01_chi_tiet_thang')
    op03a_ctv_detail.to_excel(writer, sheet_name='op03a_ctv_detail')


print('✅ Tạo thành công file ../output/ccq/cts-ccq-report.xlsx')

✅ Tạo thành công file ../output/ccq/cts-ccq-report.xlsx
